### Spark 설정하기

[+] 이미 실습 진행 했기에 스스로 코드를 채워 넣을 수 있어야 합

In [1]:
# [+] SparkConf, SparkContext 임포트
from pyspark import SparkConf, SparkContext

In [6]:
"""
    [+] SparkConf 객체 생성 및 설정
    - setMaster(), 마스터 노드 위치: "local"
    - 앱이름: "transformations_actions"
"""

conf = SparkConf().setMaster('local').setAppName('transformations_actions')

In [7]:
# [+] SparkContext 객체 생성
sc = SparkContext(conf=conf)

In [9]:
# Spark 전체 설정 값 확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1680136896873'),
 ('spark.driver.host', 'E2006009'),
 ('spark.driver.port', '54164'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.startTime', '1680136891913'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [10]:
# SpackContext 객체 하나 더 생성하기(과연?)
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=transformations_actions, master=local) created by __init__ at C:\Users\HNW\AppData\Local\Temp/ipykernel_15528/3587039525.py:2 

In [11]:
# SparkContext는 프로그램 내에서 하나의 객체만 생성할 수 있다. (= Singleton 패턴)
# SparkContext 객체 해제, 삭제하기
sc.stop()

In [13]:
# 다시 생성하기
sc = SparkContext(conf=conf)

### 기본 Actions 함수 사용하기

In [14]:
"""
    [+] List로부터 RDD 객체 생성
    - SparkContext.parallelize()
    - 리스트 값: 
    "짜장면", "마라탕", "짬뽕", 
    "떡볶이", "쌀국수", "짬뽕", 
    "짜장면", "짜장면", "짜장면", 
    "라면", "우동", "라면"
"""
lst = ["짜장면", "마라탕", "짬뽕", 
    "떡볶이", "쌀국수", "짬뽕", 
    "짜장면", "짜장면", "짜장면", 
    "라면", "우동", "라면"]
       
# [+] lst로부터 RDD 객체 생성하기
foods = sc.parallelize(lst)

In [15]:
# [+] foods RDD 내용 출력
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [17]:
# action!!
# [+] foods RDD 원소 값 별로 카운팅 countByValue
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [19]:
# take(n): 처음 n개의 원소를 출력
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [20]:
foods.take(5)

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수']

In [21]:
# first(): 첫 번째 값을 출력
foods.first()

'짜장면'

In [22]:
# count(): 원소 갯수 출력(중복된 원소 포함)
foods.count()

12

In [24]:
# distinct(): unique한 원소로 구성된 RDD를 생성하는 Transformation

unique_foods = foods.distinct()

unique_foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

### 기본 Transformations 함수 사용하기

In [25]:
# Transformations 은 지연 실행된다.
sc.parallelize([1, 2, 3]).map(lambda x: x + 2)

PythonRDD[18] at RDD at PythonRDD.scala:53

In [26]:
# Actions 를 만나면 즉시 실행된다.
sc.parallelize([1, 2, 3]).map(lambda x: x + 2).collect()

[3, 4, 5]

In [27]:
sc.parallelize([1, 2, 3]).map(lambda x: x * 2).collect()

[2, 4, 6]

In [28]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [29]:
# [+] 리스트(movies) -> RDD 객체(moviesRDD) 생성
moviesRDD = sc.parallelize(movies)

In [30]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [32]:
# 원소 갯수 유지
moviesRDD.map(lambda x: x.split(' ')).collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [33]:
# flatMap을 이용한 Tokenizing
flatMovies = moviesRDD.flatMap(lambda x: x.split(' '))

In [34]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [35]:
flatMovies.count()

21

In [38]:
"""
    [+] flatMovies 에서 '매트릭스'를 제외
    - filter(lambda ...)
    - 저장할 객체: filteredMovies
"""

filteredMovies = flatMovies.filter(lambda x: x!= '매트릭스')

In [39]:
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [40]:
"""
    집합 관련 transformations 연산
"""

num1 = sc.parallelize([1, 2, 3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [41]:
# 교집합(intersection) 구하기
num1.intersection(num2).collect()

[4]

In [43]:
# 합집합(union) 구하기
    # 중복 포합
num1.union(num2).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [44]:
# 차집합(substract) 구하기
num1.subtract(num2).collect()

[2, 1, 3]

In [45]:
num2.subtract(num1).collect()

[6, 8, 10, 5, 7, 9]

In [46]:
"""
    sample(withReplacement, fraction): 원소를 랜덤 샘플링하여 새로운 RDD를 생성하는 narrow transformation
    - withReplacement, bool[optional]: 중복 샘플링 허용 여부
    - fraction, float[optional]: 샘플링 확률[0~1], 확률이므로 정확한 사이즈가 보장되지 않음
    - seed: 결과 재현(reproducibility)을 위한 설정 값
"""

# 합집합 생성
numUnion = num1.union(num2)

# 샘플링: 중복 허용, 50% (전체에서 50%는 아님, 원소마다 sampling 확률이 50%)
numUnion.sample(True, .5).collect()

[4, 9]

In [48]:
# seed 파라미터를 설정하면 동일한 결과를 반복적으로 얻을 수 있다.
# seed 값이 동일하다면 pc 상관없이 결과 동일
numUnion.sample(True, .5, seed=1).collect()

[2, 3, 4, 6, 8, 8, 8, 8, 9, 9]

In [49]:
"""
    groupBy(): 특정 기준(함수로 정의)으로 그룹핑을 수행하는 wide transformation
"""

foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", 
                        "떡볶이", "쌀국수", "짬뽕", 
                        "짜장면", "짜장면", "짜장면",  
                        "라면", "우동", "라면", 
                        "치킨", "돈까스", "회", 
                        "햄버거", "피자"])

In [52]:
#  원소의 첫 번째 값(첫 글자)을 기준으로 그룹핑
foodsGroup = foods.groupBy(lambda x: x[0])

In [53]:
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x285c4799880>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x285c4799b20>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x285c4799a00>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x285c4799730>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x285c4799c10>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x285c4799be0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x285c4799850>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x285c4799ca0>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x285c47996d0>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x285c4799d90>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x285c4799910>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x285c4799580>)]

In [54]:
# collect() 결과를 리스트 객체(res)에 저장
res = foodsGroup.collect()

In [55]:
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x285c4725b50>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x285c47bee80>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x285c47bea90>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x285c47bec40>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x285c47be7c0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x285c47be250>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x285c47be2b0>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x285c47be3a0>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x285c47be550>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x285c47be610>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x285c47be100>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x285c47bea30>)]

In [57]:
# ResultIterable 객체는 list로 쉽게 변환 가능하다.
    # res[0][] 행 선택을 위한 index
    # res[][1] 열 선택을 위한 index
list(res[0][1])

['짜장면', '짜장면', '짜장면', '짜장면']

In [58]:
list(res[3][1])

['떡볶이']

In [59]:
# 더 이상 RDD가 아님!!
type(res)

list

In [60]:
# for 문을 이용하여 구조적으로 결과를 출력
 # list 사용이 필수!
for (k, v) in res:      # (k, v) key  value
    print(k, list(v))   # value는 list가 아니기에 변환!

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [61]:
# groupBy()는 모드 연산과 같이 수학 연산에도 사용될 수 있다.
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [62]:
# 홀수/짝수로 그룹핑
res = nums.groupBy(lambda x: x % 2).collect()

In [63]:
for (k,v) in res:
    print(k, list(v))

1 [1, 3, 5, 7, 9]
0 [2, 4, 6, 8, 10]


In [67]:
# [+] '1' 그룹에 해당되는 원소 리스트 출력하기
res = nums.groupBy(lambda x: x % 2).collect()
list(res[0][1])

[1, 3, 5, 7, 9]

In [74]:
# [+] '0' 그룹에 해당되는 원소 리스트 출력하기
res = nums.groupBy(lambda x: x % 2).collect()
list(res[1][1])

[2, 4, 6, 8, 10]